# LlamaIndex Workflow Checkpointing Demo

This notebook demonstrates the essential patterns for using LlamaIndex WorkflowCheckpointer with a simple adding numbers workflow.

## What We'll Learn
1. How to initiate a checkpointer
2. How to run a workflow
3. What checkpoint data looks like when interrupted
4. How to resume a workflow


## 1. Setup and Imports


In [ ]:
import asyncio
from typing import List, Dict, Any

from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step, Event
from llama_index.core.workflow.checkpointer import WorkflowCheckpointer


## 2. Define Event Classes

Simple events for our adding numbers workflow:


In [ ]:
class NumbersEvent(Event):
    numbers: List[int]
    message: str = ""

class SumEvent(Event):
    numbers: List[int]
    sum: int
    message: str = ""

class FinalResultEvent(Event):
    numbers: List[int]
    sum: int
    average: float
    message: str = ""


## 3. Create Simple Adding Workflow

Our workflow has 3 steps:
1. **Get Numbers** - Generate some numbers to add
2. **Calculate Sum** - Add the numbers together
3. **Calculate Average** - Find the average of the numbers


In [ ]:
class AddingWorkflow(Workflow):
    """Simple adding numbers workflow with checkpointing"""
    
    @step
    async def get_numbers(self, ev: StartEvent) -> NumbersEvent:
        """Step 1: Generate numbers to add"""
        print("🔢 Step 1: Getting numbers...")
        
        # Generate some numbers
        numbers = [1, 2, 3, 4, 5]
        
        print(f"✅ Numbers: {numbers}")
        return NumbersEvent(
            numbers=numbers,
            message=f"Generated numbers: {numbers}"
        )
    
    @step
    async def calculate_sum(self, ev: NumbersEvent) -> SumEvent:
        """Step 2: Calculate the sum"""
        print(f"➕ Step 2: Calculating sum of {ev.numbers}...")
        
        # Simulate some processing time
        await asyncio.sleep(1)
        
        total_sum = sum(ev.numbers)
        
        print(f"✅ Sum: {total_sum}")
        return SumEvent(
            numbers=ev.numbers,
            sum=total_sum,
            message=f"Sum of {ev.numbers} is {total_sum}"
        )
    
    @step
    async def calculate_average(self, ev: SumEvent) -> StopEvent:
        """Step 3: Calculate the average"""
        print(f"📊 Step 3: Calculating average...")
        
        # Simulate some processing time
        await asyncio.sleep(1)
        
        average = ev.sum / len(ev.numbers)
        
        print(f"✅ Average: {average:.2f}")
        
        return StopEvent(result={
            "numbers": ev.numbers,
            "sum": ev.sum,
            "average": round(average, 2),
            "message": f"Numbers: {ev.numbers}, Sum: {ev.sum}, Average: {average:.2f}"
        })
